In [1]:
import googleapiclient.discovery
import googleapiclient.errors
import numpy as np
import pandas as pd
import glob
import nltk 
import os
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import re
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.pipeline import Pipeline
from langdetect import detect
import shutil
import random
from wordcloud import WordCloud
import gensim
import seaborn as sns
from textblob import TextBlob
from tabulate import tabulate
from sklearn.model_selection import GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.datasets import load_files
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split


nltk.download('vader_lexicon')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Archisa\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Archisa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
train_df = pd.read_csv('train.csv.zip')
test_df = pd.read_csv('test.csv.zip')

In [3]:
train_df = train_df.head(10000)

In [4]:
train_df.shape

(10000, 2)

In [5]:
def remove_punctuation(text):
    # Define the pattern to match punctuation
    punctuation_pattern = r'[^\w\s]'
    # Replace punctuation with an empty string
    text_without_punctuation = re.sub(punctuation_pattern, '', text)
    # Normalize whitespace
    normalized_text = re.sub(r'\s+', ' ', text_without_punctuation)
    return normalized_text

# Text Processing
def preprocess_text(text, method='snowballstemming'):
    # Tokenize and lowercase
    tokens = word_tokenize(text.lower())

    # Define the set of stopwords
    stop_words = set(stopwords.words('english'))
    
    # Remove stopwords and punctuation
    tokens = [remove_punctuation(token) for token in tokens if token.isalnum() and token not in stop_words]
    
    if method == 'snowballstemming':
        stemmer = nltk.stem.SnowballStemmer('english')
        tokens = [stemmer.stem(token) for token in tokens]

    elif method == 'porterstemming':
        stemmer = nltk.stem.PorterStemmer()
        tokens = [stemmer.stem(token) for token in tokens]

    elif method == 'lemmatization':
        lemmatizer = nltk.stem.WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return ' '.join(tokens)

In [6]:
train_df['ssprocessing_text'] = train_df['Review'].apply(preprocess_text, method='snowballstemming')
train_df['psprocessing_text'] = train_df['Review'].apply(preprocess_text, method='porterstemming')
train_df['lemprocessing_text'] = train_df['Review'].apply(preprocess_text, method='lemmatization')

In [7]:
train_df

,overall,Review,ssprocessing_text,psprocessing_text,lemprocessing_text
0,5,I love these glitter pens. They sparkle deligh...,love glitter pen sparkl delight page brilliant...,love glitter pen sparkl delight page brilliant...,love glitter pen sparkle delightfully page bri...
1,5,It works well with my machine. I use mostly c...,work well machin use most cone,work well machin use mostli cone,work well machine use mostly cone
2,5,"This is a great assortment of colors, though t...",great assort color though lot pink mix still c...,great assort color though lot pink mix still c...,great assortment color though lot pink mix sti...
3,5,Just what I was looking for.,look,look,looking
4,5,I make 400 birds for the hospital each month.,make 400 bird hospit month,make 400 bird hospit month,make 400 bird hospital month
...,...,...,...,...,...
9995,5,"Lovely yarn, very fast delivery, I am so pleas...",love yarn fast deliveri pleas buy,love yarn fast deliveri pleas buy,lovely yarn fast delivery pleased buy
9996,5,These perfectly match some charms I ordered to...,perfect match charm order make photo pendant like,perfectli match charm order make photo pendant...,perfectly match charm ordered make photo penda...
9997,5,Perfect for my project.,perfect project,perfect project,perfect project
9998,5,Great product!,great product,great product,great product


In [8]:
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense
from sklearn.model_selection import train_test_split

import gensim

In [9]:
# tokenized_df = train_df.copy()

# # Iterate over each column
# for column in train_df.columns:
#     # Check if the column is not 'overall'
#     if column != 'overall':
#         # Tokenize values in the column
#         tokenized_values = train_df[column].apply(lambda x: word_tokenize(x))
#         # Add tokenized values to the new DataFrame
#         tokenized_df[column] = tokenized_values

# tokenized_df

# word2vec_model = gensim.models.Word2Vec(subset_df["tokens"].tolist(), min_count=5, window=9, vector_size=100)

In [10]:
train_df_ss = train_df[['overall', 'ssprocessing_text']].copy()
train_df_ss

,overall,ssprocessing_text
0,5,love glitter pen sparkl delight page brilliant...
1,5,work well machin use most cone
2,5,great assort color though lot pink mix still c...
3,5,look
4,5,make 400 bird hospit month
...,...,...
9995,5,love yarn fast deliveri pleas buy
9996,5,perfect match charm order make photo pendant like
9997,5,perfect project
9998,5,great product


In [11]:
train_df_ps = train_df[['overall', 'psprocessing_text']].copy()
train_df_ps

,overall,psprocessing_text
0,5,love glitter pen sparkl delight page brilliant...
1,5,work well machin use mostli cone
2,5,great assort color though lot pink mix still c...
3,5,look
4,5,make 400 bird hospit month
...,...,...
9995,5,love yarn fast deliveri pleas buy
9996,5,perfectli match charm order make photo pendant...
9997,5,perfect project
9998,5,great product


In [12]:
train_df_lem = train_df[['overall', 'lemprocessing_text']].copy()
train_df_lem

,overall,lemprocessing_text
0,5,love glitter pen sparkle delightfully page bri...
1,5,work well machine use mostly cone
2,5,great assortment color though lot pink mix sti...
3,5,looking
4,5,make 400 bird hospital month
...,...,...
9995,5,lovely yarn fast delivery pleased buy
9996,5,perfectly match charm ordered make photo penda...
9997,5,perfect project
9998,5,great product


In [13]:
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

def early_stopping(monitor='val_loss', min_delta=0, patience=5, mode='auto'):
    return EarlyStopping(monitor=monitor, min_delta=min_delta, patience=patience, mode=mode)

# Function to create step decay learning rate scheduler
def step_decay(initial_lr=0.001, drop_factor=0.5, epochs_drop=5):
    def scheduler(epoch):
        return initial_lr * np.power(drop_factor, np.floor((1 + epoch) / epochs_drop))
    return scheduler

### LSTM Lemmatising

In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Split the data into features and target labels
X = train_df['lemprocessing_text']
y = train_df['overall']

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)

# Padding sequences
max_len = 100  # Define your maximum sequence length
X_padded = pad_sequences(X_sequences, maxlen=max_len)

# Convert target labels to numerical values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

# Define LSTM model
def LSTM_model(input_length, vocab_size, embedding_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_size, input_length=input_length))
    model.add(LSTM(100))
    model.add(Dropout(0.2))
    model.add(Dense(5, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Define parameters
vocab_size = len(tokenizer.word_index) + 1  # Add 1 for padding token
embedding_size = 100  # Define your embedding size

# Create and compile the LSTM model
lstm_model = LSTM_model(max_len, vocab_size, embedding_size)

# Early stopping and decay callbacks
early_stop_callback = early_stopping(patience=5)
decay = LearningRateScheduler(step_decay(initial_lr=0.001, drop_factor=0.5, epochs_drop=5))
callbacks_list = [early_stop_callback, decay]

# Train the model
lstm_model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), callbacks=callbacks_list)

# Evaluate the model
loss, accuracy = lstm_model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)


Epoch 1/10
125/125 [==============================] - 35s 257ms/step - loss: 0.8419 - accuracy: 0.7635 - val_loss: 0.7108 - val_accuracy: 0.7800 - lr: 0.0010
Epoch 2/10
125/125 [==============================] - 30s 238ms/step - loss: 0.6401 - accuracy: 0.7790 - val_loss: 0.6854 - val_accuracy: 0.7750 - lr: 0.0010
Epoch 3/10
125/125 [==============================] - 31s 248ms/step - loss: 0.5275 - accuracy: 0.8126 - val_loss: 0.7149 - val_accuracy: 0.7720 - lr: 0.0010
Epoch 4/10
125/125 [==============================] - 29s 234ms/step - loss: 0.4330 - accuracy: 0.8465 - val_loss: 0.7334 - val_accuracy: 0.7655 - lr: 0.0010
Epoch 5/10
125/125 [==============================] - 31s 248ms/step - loss: 0.3445 - accuracy: 0.8839 - val_loss: 0.8736 - val_accuracy: 0.7565 - lr: 5.0000e-04
Epoch 6/10
125/125 [==============================] - 31s 251ms/step - loss: 0.2979 - accuracy: 0.9018 - val_loss: 0.9460 - val_accuracy: 0.7320 - lr: 5.0000e-04
Epoch 7/10
63/63 [==========================

In [20]:
# Define LSTM model
def LSTM_model2(input_length, vocab_size, embedding_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_size, input_length=input_length))
    model.add(LSTM(150))
    model.add(Dropout(0.3))
    model.add(Dense(5, activation='relu'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adagrad', metrics=['accuracy'])
    return model

# Create and compile the LSTM model
lstm_model2 = LSTM_model2(max_len, vocab_size, embedding_size)

# Train the model
lstm_model2.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), callbacks=callbacks_list)

# Evaluate the model
loss, accuracy = lstm_model2.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)


Epoch 1/10
125/125 [==============================] - 30s 220ms/step - loss: 2.1485 - accuracy: 0.7219 - val_loss: 1.9017 - val_accuracy: 0.7015 - lr: 0.0010
Epoch 2/10
125/125 [==============================] - 26s 212ms/step - loss: 1.9187 - accuracy: 0.7293 - val_loss: 1.7563 - val_accuracy: 0.7680 - lr: 0.0010
Epoch 3/10
125/125 [==============================] - 27s 212ms/step - loss: 1.9050 - accuracy: 0.7444 - val_loss: 1.7790 - val_accuracy: 0.7715 - lr: 0.0010
Epoch 4/10
125/125 [==============================] - 27s 219ms/step - loss: 1.8972 - accuracy: 0.7458 - val_loss: 1.6910 - val_accuracy: 0.7695 - lr: 0.0010
Epoch 5/10
125/125 [==============================] - 33s 261ms/step - loss: 1.7723 - accuracy: 0.7425 - val_loss: 1.6300 - val_accuracy: 0.7440 - lr: 5.0000e-04
Epoch 6/10
125/125 [==============================] - 32s 260ms/step - loss: 1.6721 - accuracy: 0.7249 - val_loss: 1.5184 - val_accuracy: 0.7610 - lr: 5.0000e-04
Epoch 7/10
125/125 [========================

### LSTM with Porter Stemmer

In [32]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Split the data into features and target labels
X = train_df['psprocessing_text']
y = train_df['overall']

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)

# Padding sequences
max_len = 100  # Define your maximum sequence length
X_padded = pad_sequences(X_sequences, maxlen=max_len)

# Convert target labels to numerical values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

# Define LSTM model
def LSTM_model_ps(input_length, vocab_size, embedding_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_size, input_length=input_length))
    model.add(LSTM(100))
    model.add(Dropout(0.2))
    model.add(Dense(5, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Define parameters
vocab_size = len(tokenizer.word_index) + 1  # Add 1 for padding token
embedding_size = 100  # Define your embedding size

# Create and compile the LSTM model
lstm_model_ps = LSTM_model_ps(max_len, vocab_size, embedding_size)

# Early stopping and decay callbacks
early_stop_callback = early_stopping(patience=5)
decay = LearningRateScheduler(step_decay(initial_lr=0.001, drop_factor=0.5, epochs_drop=5))
callbacks_list = [early_stop_callback, decay]

# Train the model
lstm_model_ps.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), callbacks=callbacks_list)

# Evaluate the model
loss, accuracy = lstm_model_ps.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)


Epoch 1/10
125/125 [==============================] - 37s 268ms/step - loss: 0.8415 - accuracy: 0.7602 - val_loss: 0.7244 - val_accuracy: 0.7815 - lr: 0.0010
Epoch 2/10
125/125 [==============================] - 34s 276ms/step - loss: 0.6368 - accuracy: 0.7769 - val_loss: 0.6988 - val_accuracy: 0.7685 - lr: 0.0010
Epoch 3/10
125/125 [==============================] - 36s 284ms/step - loss: 0.5485 - accuracy: 0.8051 - val_loss: 0.7050 - val_accuracy: 0.7735 - lr: 0.0010
Epoch 4/10
125/125 [==============================] - 39s 310ms/step - loss: 0.4695 - accuracy: 0.8340 - val_loss: 0.7575 - val_accuracy: 0.7635 - lr: 0.0010
Epoch 5/10
125/125 [==============================] - 39s 313ms/step - loss: 0.3863 - accuracy: 0.8686 - val_loss: 0.7594 - val_accuracy: 0.7625 - lr: 5.0000e-04
Epoch 6/10
125/125 [==============================] - 35s 283ms/step - loss: 0.3440 - accuracy: 0.8831 - val_loss: 0.8294 - val_accuracy: 0.7605 - lr: 5.0000e-04
Epoch 7/10
63/63 [==========================

In [33]:
# Define LSTM model
def LSTM_model2_ps(input_length, vocab_size, embedding_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_size, input_length=input_length))
    model.add(LSTM(150))
    model.add(Dropout(0.3))
    model.add(Dense(5, activation='relu'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adagrad', metrics=['accuracy'])
    return model

# Create and compile the LSTM model
lstm_model2_ps = LSTM_model2_ps(max_len, vocab_size, embedding_size)

# Train the model
lstm_model2_ps.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), callbacks=callbacks_list)

# Evaluate the model
loss, accuracy = lstm_model2_ps.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)


Epoch 1/10
125/125 [==============================] - 51s 338ms/step - loss: 1.6742 - accuracy: 0.6929 - val_loss: 1.3174 - val_accuracy: 0.7635 - lr: 0.0010
Epoch 2/10
125/125 [==============================] - 36s 285ms/step - loss: 1.2595 - accuracy: 0.7534 - val_loss: 1.1400 - val_accuracy: 0.7780 - lr: 0.0010
Epoch 3/10
125/125 [==============================] - 34s 273ms/step - loss: 1.1417 - accuracy: 0.7628 - val_loss: 1.0605 - val_accuracy: 0.7790 - lr: 0.0010
Epoch 4/10
125/125 [==============================] - 33s 262ms/step - loss: 1.0983 - accuracy: 0.7656 - val_loss: 0.9627 - val_accuracy: 0.7800 - lr: 0.0010
Epoch 5/10
125/125 [==============================] - 33s 263ms/step - loss: 1.0249 - accuracy: 0.7660 - val_loss: 0.9146 - val_accuracy: 0.7800 - lr: 5.0000e-04
Epoch 6/10
125/125 [==============================] - 32s 256ms/step - loss: 1.0078 - accuracy: 0.7663 - val_loss: 0.9288 - val_accuracy: 0.7800 - lr: 5.0000e-04
Epoch 7/10
125/125 [========================

### LSTM with Snowball Stemmer

In [34]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Split the data into features and target labels
X = train_df['ssprocessing_text']
y = train_df['overall']

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)

# Padding sequences
max_len = 100  # Define your maximum sequence length
X_padded = pad_sequences(X_sequences, maxlen=max_len)

# Convert target labels to numerical values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

# Define LSTM model
def LSTM_model_ss(input_length, vocab_size, embedding_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_size, input_length=input_length))
    model.add(LSTM(100))
    model.add(Dropout(0.2))
    model.add(Dense(5, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Define parameters
vocab_size = len(tokenizer.word_index) + 1  # Add 1 for padding token
embedding_size = 100  # Define your embedding size

# Create and compile the LSTM model
lstm_model_ss = LSTM_model_ss(max_len, vocab_size, embedding_size)

# Early stopping and decay callbacks
early_stop_callback = early_stopping(patience=5)
decay = LearningRateScheduler(step_decay(initial_lr=0.001, drop_factor=0.5, epochs_drop=5))
callbacks_list = [early_stop_callback, decay]

# Train the model
lstm_model_ss.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), callbacks=callbacks_list)

# Evaluate the model
loss, accuracy = lstm_model_ss.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)


Epoch 1/10
125/125 [==============================] - 43s 315ms/step - loss: 0.8335 - accuracy: 0.7598 - val_loss: 0.6870 - val_accuracy: 0.7810 - lr: 0.0010
Epoch 2/10
125/125 [==============================] - 42s 334ms/step - loss: 0.6322 - accuracy: 0.7778 - val_loss: 0.6746 - val_accuracy: 0.7745 - lr: 0.0010
Epoch 3/10
125/125 [==============================] - 40s 323ms/step - loss: 0.5433 - accuracy: 0.8049 - val_loss: 0.6884 - val_accuracy: 0.7595 - lr: 0.0010
Epoch 4/10
125/125 [==============================] - 37s 293ms/step - loss: 0.4672 - accuracy: 0.8380 - val_loss: 0.7139 - val_accuracy: 0.7635 - lr: 0.0010
Epoch 5/10
125/125 [==============================] - 36s 289ms/step - loss: 0.3869 - accuracy: 0.8635 - val_loss: 0.8104 - val_accuracy: 0.7495 - lr: 5.0000e-04
Epoch 6/10
125/125 [==============================] - 30s 243ms/step - loss: 0.3475 - accuracy: 0.8802 - val_loss: 0.8480 - val_accuracy: 0.7455 - lr: 5.0000e-04
Epoch 7/10
63/63 [==========================

In [35]:
# Define LSTM model
def LSTM_model2_ss(input_length, vocab_size, embedding_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_size, input_length=input_length))
    model.add(LSTM(150))
    model.add(Dropout(0.3))
    model.add(Dense(5, activation='relu'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adagrad', metrics=['accuracy'])
    return model

# Create and compile the LSTM model
lstm_model2_ss = LSTM_model2_ss(max_len, vocab_size, embedding_size)

# Train the model
lstm_model2_ss.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), callbacks=callbacks_list)

# Evaluate the model
loss, accuracy = lstm_model2_ss.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)


Epoch 1/10
125/125 [==============================] - 36s 262ms/step - loss: 1.6636 - accuracy: 0.7376 - val_loss: 1.3586 - val_accuracy: 0.7790 - lr: 0.0010
Epoch 2/10
125/125 [==============================] - 34s 270ms/step - loss: 1.3196 - accuracy: 0.7589 - val_loss: 1.0429 - val_accuracy: 0.7780 - lr: 0.0010
Epoch 3/10
125/125 [==============================] - 32s 258ms/step - loss: 1.1402 - accuracy: 0.7581 - val_loss: 0.9925 - val_accuracy: 0.7790 - lr: 0.0010
Epoch 4/10
125/125 [==============================] - 31s 251ms/step - loss: 1.0932 - accuracy: 0.7641 - val_loss: 0.9945 - val_accuracy: 0.7800 - lr: 0.0010
Epoch 5/10
125/125 [==============================] - 32s 252ms/step - loss: 1.0802 - accuracy: 0.7639 - val_loss: 0.9682 - val_accuracy: 0.7800 - lr: 5.0000e-04
Epoch 6/10
125/125 [==============================] - 32s 255ms/step - loss: 1.0356 - accuracy: 0.7635 - val_loss: 0.9279 - val_accuracy: 0.7800 - lr: 5.0000e-04
Epoch 7/10
125/125 [========================

### LSTM + Word2Vec with Lemmatising

In [24]:
from gensim.models import Word2Vec


# Split the data into features and target labels
X = train_df['lemprocessing_text']
y = train_df['overall']

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)

# Padding sequences
max_len = 100  # Define your maximum sequence length
X_padded = pad_sequences(X_sequences, maxlen=max_len)

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=X, vector_size=embedding_size, window=5, min_count=1, workers=4)

# Convert words to Word2Vec embeddings
word_index = tokenizer.word_index
embedding_matrix = np.zeros((vocab_size, embedding_size))
# Convert words to Word2Vec embeddings
embedding_matrix = np.zeros((vocab_size, embedding_size))
for word, i in word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]


# Convert target labels to numerical values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

# Define LSTM model with Word2Vec embeddings
def LSTM_model_with_Word2Vec(input_length, vocab_size, embedding_size, embedding_matrix):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_size, input_length=input_length))
    model.add(LSTM(150))
    model.add(Dropout(0.3))
    model.add(Dense(5, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Define parameters
vocab_size = len(tokenizer.word_index) + 1  # Add 1 for padding token
embedding_size = 100  # Define your embedding size

# Create and compile the LSTM model with Word2Vec embeddings
lstm_model_with_word2vec = LSTM_model_with_Word2Vec(max_len, vocab_size, embedding_size, embedding_matrix)

# Train the model
lstm_model_with_word2vec.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), callbacks=callbacks_list)

# Evaluate the model
loss, accuracy = lstm_model_with_word2vec.evaluate(X_test, y_test)
print("Test Accuracy with Word2Vec embeddings:", accuracy)


Epoch 1/10
125/125 [==============================] - 37s 275ms/step - loss: 0.8359 - accuracy: 0.7630 - val_loss: 0.6983 - val_accuracy: 0.7800 - lr: 0.0010
Epoch 2/10
125/125 [==============================] - 32s 255ms/step - loss: 0.6254 - accuracy: 0.7797 - val_loss: 0.6832 - val_accuracy: 0.7695 - lr: 0.0010
Epoch 3/10
125/125 [==============================] - 33s 264ms/step - loss: 0.5136 - accuracy: 0.8194 - val_loss: 0.7209 - val_accuracy: 0.7755 - lr: 0.0010
Epoch 4/10
125/125 [==============================] - 36s 289ms/step - loss: 0.4267 - accuracy: 0.8482 - val_loss: 0.7996 - val_accuracy: 0.7655 - lr: 0.0010
Epoch 5/10
125/125 [==============================] - 40s 323ms/step - loss: 0.3263 - accuracy: 0.8870 - val_loss: 0.8729 - val_accuracy: 0.7575 - lr: 5.0000e-04
Epoch 6/10
125/125 [==============================] - 35s 282ms/step - loss: 0.2877 - accuracy: 0.9021 - val_loss: 0.9408 - val_accuracy: 0.7515 - lr: 5.0000e-04
Epoch 7/10
63/63 [==========================

In [26]:
# Define LSTM model with Word2Vec embeddings
def LSTM_model2_with_Word2Vec(input_length, vocab_size, embedding_size, embedding_matrix):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_size, input_length=input_length))
    model.add(LSTM(200))
    model.add(Dropout(0.35))
    model.add(Dense(5, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create and compile the LSTM model with Word2Vec embeddings
lstm_model2_with_word2vec = LSTM_model2_with_Word2Vec(max_len, vocab_size, embedding_size, embedding_matrix)

# Train the model
lstm_model2_with_word2vec.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test), callbacks=callbacks_list)

# Evaluate the model
loss, accuracy = lstm_model2_with_word2vec.evaluate(X_test, y_test)
print("Test Accuracy with Word2Vec embeddings:", accuracy)


Epoch 1/20
125/125 [==============================] - 54s 403ms/step - loss: 0.8072 - accuracy: 0.7607 - val_loss: 0.6757 - val_accuracy: 0.7800 - lr: 0.0010
Epoch 2/20
125/125 [==============================] - 51s 412ms/step - loss: 0.6132 - accuracy: 0.7807 - val_loss: 0.6791 - val_accuracy: 0.7735 - lr: 0.0010
Epoch 3/20
125/125 [==============================] - 50s 397ms/step - loss: 0.5143 - accuracy: 0.8154 - val_loss: 0.7231 - val_accuracy: 0.7570 - lr: 0.0010
Epoch 4/20
125/125 [==============================] - 44s 349ms/step - loss: 0.4309 - accuracy: 0.8464 - val_loss: 0.8451 - val_accuracy: 0.7395 - lr: 0.0010
Epoch 5/20
125/125 [==============================] - 44s 349ms/step - loss: 0.3334 - accuracy: 0.8836 - val_loss: 0.8849 - val_accuracy: 0.7490 - lr: 5.0000e-04
Epoch 6/20
63/63 [==============================] - 5s 81ms/step - loss: 0.9231 - accuracy: 0.7535
Test Accuracy with Word2Vec embeddings: 0.7534999847412109


### LSTM + Word2Vec with Porter Stemmer

In [27]:
# Split the data into features and target labels
X = train_df['psprocessing_text']
y = train_df['overall']

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)

# Padding sequences
max_len = 100  # Define your maximum sequence length
X_padded = pad_sequences(X_sequences, maxlen=max_len)

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=X, vector_size=embedding_size, window=5, min_count=1, workers=4)

# Convert words to Word2Vec embeddings
word_index = tokenizer.word_index
embedding_matrix = np.zeros((vocab_size, embedding_size))
# Convert words to Word2Vec embeddings
embedding_matrix = np.zeros((vocab_size, embedding_size))
for word, i in word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]


# Convert target labels to numerical values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

# Define LSTM model with Word2Vec embeddings
def LSTM_model_with_Word2Vec_ps(input_length, vocab_size, embedding_size, embedding_matrix):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_size, input_length=input_length))
    model.add(LSTM(150))
    model.add(Dropout(0.3))
    model.add(Dense(5, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Define parameters
vocab_size = len(tokenizer.word_index) + 1  # Add 1 for padding token
embedding_size = 100  # Define your embedding size

# Create and compile the LSTM model with Word2Vec embeddings
lstm_model_with_word2vec_ps = LSTM_model_with_Word2Vec_ps(max_len, vocab_size, embedding_size, embedding_matrix)

# Train the model
lstm_model_with_word2vec_ps.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), callbacks=callbacks_list)

# Evaluate the model
loss, accuracy = lstm_model_with_word2vec_ps.evaluate(X_test, y_test)
print("Test Accuracy with Word2Vec embeddings:", accuracy)


Epoch 1/10
125/125 [==============================] - 56s 402ms/step - loss: 0.8222 - accuracy: 0.7631 - val_loss: 0.7021 - val_accuracy: 0.7805 - lr: 0.0010
Epoch 2/10
125/125 [==============================] - 45s 364ms/step - loss: 0.6362 - accuracy: 0.7780 - val_loss: 0.6726 - val_accuracy: 0.7750 - lr: 0.0010
Epoch 3/10
125/125 [==============================] - 46s 366ms/step - loss: 0.5507 - accuracy: 0.8020 - val_loss: 0.6895 - val_accuracy: 0.7725 - lr: 0.0010
Epoch 4/10
125/125 [==============================] - 46s 366ms/step - loss: 0.4761 - accuracy: 0.8305 - val_loss: 0.7235 - val_accuracy: 0.7645 - lr: 0.0010
Epoch 5/10
125/125 [==============================] - 46s 370ms/step - loss: 0.3961 - accuracy: 0.8648 - val_loss: 0.7640 - val_accuracy: 0.7550 - lr: 5.0000e-04
Epoch 6/10
125/125 [==============================] - 47s 380ms/step - loss: 0.3537 - accuracy: 0.8800 - val_loss: 0.8138 - val_accuracy: 0.7635 - lr: 5.0000e-04
Epoch 7/10
63/63 [==========================

In [29]:
def LSTM_model2_with_Word2Vec_ps(input_length, vocab_size, embedding_size, embedding_matrix):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_size, input_length=input_length))
    model.add(LSTM(200))
    model.add(Dropout(0.35))
    model.add(Dense(5, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create and compile the LSTM model with Word2Vec embeddings
lstm_model2_with_word2vec_ps = LSTM_model2_with_Word2Vec_ps(max_len, vocab_size, embedding_size, embedding_matrix)

# Train the model
lstm_model2_with_word2vec_ps.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), callbacks=callbacks_list)

# Evaluate the model
loss, accuracy = lstm_model2_with_word2vec_ps.evaluate(X_test, y_test)
print("Test Accuracy with Word2Vec embeddings:", accuracy)

Epoch 1/20
250/250 [==============================] - 91s 353ms/step - loss: 0.7746 - accuracy: 0.7639 - val_loss: 0.6703 - val_accuracy: 0.7810 - lr: 0.0010
Epoch 2/20
250/250 [==============================] - 90s 359ms/step - loss: 0.6179 - accuracy: 0.7839 - val_loss: 0.6854 - val_accuracy: 0.7710 - lr: 0.0010
Epoch 3/20
250/250 [==============================] - 113s 455ms/step - loss: 0.5237 - accuracy: 0.8123 - val_loss: 0.6996 - val_accuracy: 0.7605 - lr: 0.0010
Epoch 4/20
250/250 [==============================] - 120s 480ms/step - loss: 0.4470 - accuracy: 0.8428 - val_loss: 0.7385 - val_accuracy: 0.7660 - lr: 0.0010
Epoch 5/20
250/250 [==============================] - 111s 445ms/step - loss: 0.3487 - accuracy: 0.8819 - val_loss: 0.8479 - val_accuracy: 0.7515 - lr: 5.0000e-04
Epoch 6/20
63/63 [==============================] - 8s 119ms/step - loss: 0.9272 - accuracy: 0.7450
Test Accuracy with Word2Vec embeddings: 0.7450000047683716


### LSTM + Word2Vec with Snowball Stemmer

In [30]:
# Split the data into features and target labels
X = train_df['ssprocessing_text']
y = train_df['overall']

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)

# Padding sequences
max_len = 100  # Define your maximum sequence length
X_padded = pad_sequences(X_sequences, maxlen=max_len)

# Train Word2Vec model
word2vec_model = Word2Vec(sentences=X, vector_size=embedding_size, window=5, min_count=1, workers=4)

# Convert words to Word2Vec embeddings
word_index = tokenizer.word_index
embedding_matrix = np.zeros((vocab_size, embedding_size))
# Convert words to Word2Vec embeddings
embedding_matrix = np.zeros((vocab_size, embedding_size))
for word, i in word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[i] = word2vec_model.wv[word]


# Convert target labels to numerical values
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_encoded, test_size=0.2, random_state=42)

# Define LSTM model with Word2Vec embeddings
def LSTM_model_with_Word2Vec_ss(input_length, vocab_size, embedding_size, embedding_matrix):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_size, input_length=input_length))
    model.add(LSTM(150))
    model.add(Dropout(0.3))
    model.add(Dense(5, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Define parameters
vocab_size = len(tokenizer.word_index) + 1  # Add 1 for padding token
embedding_size = 100  # Define your embedding size

# Create and compile the LSTM model with Word2Vec embeddings
lstm_model_with_word2vec_ss = LSTM_model_with_Word2Vec_ss(max_len, vocab_size, embedding_size, embedding_matrix)

# Train the model
lstm_model_with_word2vec_ss.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), callbacks=callbacks_list)

# Evaluate the model
loss, accuracy = lstm_model_with_word2vec_ss.evaluate(X_test, y_test)
print("Test Accuracy with Word2Vec embeddings:", accuracy)


Epoch 1/10
125/125 [==============================] - 47s 346ms/step - loss: 0.8480 - accuracy: 0.7620 - val_loss: 0.7066 - val_accuracy: 0.7810 - lr: 0.0010
Epoch 2/10
125/125 [==============================] - 47s 375ms/step - loss: 0.6386 - accuracy: 0.7768 - val_loss: 0.7015 - val_accuracy: 0.7740 - lr: 0.0010
Epoch 3/10
125/125 [==============================] - 49s 393ms/step - loss: 0.5455 - accuracy: 0.8040 - val_loss: 0.6973 - val_accuracy: 0.7680 - lr: 0.0010
Epoch 4/10
125/125 [==============================] - 49s 394ms/step - loss: 0.4706 - accuracy: 0.8366 - val_loss: 0.7379 - val_accuracy: 0.7850 - lr: 0.0010
Epoch 5/10
125/125 [==============================] - 53s 423ms/step - loss: 0.3821 - accuracy: 0.8690 - val_loss: 0.8066 - val_accuracy: 0.7515 - lr: 5.0000e-04
Epoch 6/10
125/125 [==============================] - 45s 357ms/step - loss: 0.3397 - accuracy: 0.8852 - val_loss: 0.8224 - val_accuracy: 0.7640 - lr: 5.0000e-04
Epoch 7/10
125/125 [========================

In [31]:
def LSTM_model2_with_Word2Vec_ss(input_length, vocab_size, embedding_size, embedding_matrix):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_size, input_length=input_length))
    model.add(LSTM(200))
    model.add(Dropout(0.35))
    model.add(Dense(5, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Create and compile the LSTM model with Word2Vec embeddings
lstm_model2_with_word2vec_ss = LSTM_model2_with_Word2Vec_ss(max_len, vocab_size, embedding_size, embedding_matrix)

# Train the model
lstm_model2_with_word2vec_ss.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), callbacks=callbacks_list)

# Evaluate the model
loss, accuracy = lstm_model2_with_word2vec_ss.evaluate(X_test, y_test)
print("Test Accuracy with Word2Vec embeddings:", accuracy)

Epoch 1/20
250/250 [==============================] - 92s 353ms/step - loss: 0.7863 - accuracy: 0.7659 - val_loss: 0.6774 - val_accuracy: 0.7780 - lr: 0.0010
Epoch 2/20
250/250 [==============================] - 112s 448ms/step - loss: 0.6183 - accuracy: 0.7831 - val_loss: 0.6760 - val_accuracy: 0.7785 - lr: 0.0010
Epoch 3/20
250/250 [==============================] - 107s 427ms/step - loss: 0.5128 - accuracy: 0.8176 - val_loss: 0.6955 - val_accuracy: 0.7640 - lr: 0.0010
Epoch 4/20
250/250 [==============================] - 93s 373ms/step - loss: 0.4344 - accuracy: 0.8494 - val_loss: 0.7311 - val_accuracy: 0.7610 - lr: 0.0010
Epoch 5/20
250/250 [==============================] - 89s 356ms/step - loss: 0.3461 - accuracy: 0.8815 - val_loss: 0.7804 - val_accuracy: 0.7550 - lr: 5.0000e-04
Epoch 6/20
250/250 [==============================] - 109s 437ms/step - loss: 0.2967 - accuracy: 0.9016 - val_loss: 0.8429 - val_accuracy: 0.7450 - lr: 5.0000e-04
Epoch 7/20
63/63 [=======================